In [ ]:
from WindPy import *
w.start()


import os
import glob
import time
import datetime
import pandas as pd
pd.set_option('display.max_columns',200)
pd.options.mode.chained_assignment = None
import numpy as np

database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

il = pd.read_csv(r'D:\work\project 17 AMAC\tickStockList_AMAC.csv')
il['StockID'] = il['StockID'].str[3:] + '.CSI'
il = il['StockID'].values

import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz
import TSLPy3

def updateAShare(date):
    TRDate = str(date)
    tsstr = """
           BegT:=%s;
           EndT:=%s;
           SetSysParam(pn_stock(),'SH000001');
           SetSysParam(PN_Cycle(),cy_day());
           dateArr:=MarketTradeDayQk(BegT,EndT);
           r:=array();
           for nI:=0 to length(dateArr)-1 do
           begin
             echo dateArr[nI];
             t:= getabkbydate('A股',dateArr[nI]);
             r:=r union2 t;
           end;
           r:= select [0] as 'StockID' from `r end;
           r := select * from r order by ['StockID'] end;
           return r;
            """%(TRDate + 'T', TRDate + 'T + 0.99')
    stockList = pd.DataFrame(TSLPy3.RemoteExecute(tsstr,[],{})[1])
    stockList.columns = list(pd.Series(stockList.columns).str.decode('GBK'))
    stockList['StockID'] = stockList['StockID'].str.decode('GBK')
    stockList['skey'] = np.where(stockList['StockID'].str[:2] == 'SH', 1000000 + stockList['StockID'].str[2:].astype(int),
                                2000000 + stockList['StockID'].str[2:].astype(int))
    stockList['date'] = int(TRDate)
    return stockList

dl = [20201113, 20201116, 20201117, 20201118, 20201119]
total_stock = []
for d in dl:
    data = updateAShare(d)
    total_stock += [data]
total_stock = pd.concat(total_stock, sort=False)

data2 = []
add = []
startTm = datetime.datetime.now()

data2 = []
for i in il:
    data = w.wset("indexconstituent","date=%s; windcode=%s"%(d, i))
    df = pd.DataFrame(data=np.array(data.Data).T, columns=data.Fields)
    df['index_id'] = 3000000 + int(i[1:6])
    try:
        assert(abs(df['i_weight'].sum() - 100) < 0.1)
    except:
        print('sum of weight far from 100')
        print(df['i_weight'].sum())
        print(df)
    df['skey'] = df['wind_code'].str[:-3].astype(int)
    df['skey'] = np.where(df['skey'] < 600000, df['skey'] + 2000000, df['skey'] + 1000000)
    df['date'] = df['date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
    corrections = {3011030: 'AMAC 农林',    
           3011031: 'AMAC 采矿', 
           3011041: 'AMAC 公用', 
           3011042: 'AMAC 建筑', 
           3011043: 'AMAC 交运',    
           3011044: 'AMAC 信息', 
           3011045: 'AMAC 批零', 
           3011046: 'AMAC 金融', 
           3011047: 'AMAC 地产',    
           3011049: 'AMAC 文体', 
           3011050: 'AMAC 综企', 
           3030036: 'AMAC 餐饮',  
           3030037: 'AMAC 商务',    
           3030038: 'AMAC 科技', 
           3030039: 'AMAC 公共', 
           3030040: 'AMAC 社会',
           3030041: 'AMAC 农副', 
           3030042: 'AMAC 食品', 
           3030043: 'AMAC 饮料', 
           3030044: 'AMAC 纺织',    
           3030045: 'AMAC 服装', 
           3030046: 'AMAC 皮革', 
           3030047: 'AMAC 木材', 
           3030048: 'AMAC 家具',    
           3030049: 'AMAC 造纸', 
           3030050: 'AMAC 印刷', 
           3030051: 'AMAC 文教',  
           3030052: 'AMAC 石化',    
           3030053: 'AMAC 化学', 
           3030054: 'AMAC 医药', 
           3030055: 'AMAC 化纤',  
           3030056: 'AMAC 橡胶', 
           3030057: 'AMAC 矿物', 
           3030058: 'AMAC 钢铁', 
           3030059: 'AMAC 有色',    
           3030060: 'AMAC 金属', 
           3030061: 'AMAC 通用', 
           3030062: 'AMAC 专用', 
           3030063: 'AMAC 汽车',    
           3030064: 'AMAC 运输', 
           3030065: 'AMAC 电气', 
           3030066: 'AMAC 电子',  
           3030067: 'AMAC 仪表'}
    df['index_name'] = df['index_id']
    df.index_name = df.index_name.map(corrections)
    data2 += [df]
data2 = pd.concat(data2).reset_index(drop=True)
stock_list = list(set(total_stock[total_stock['date'] == d]['skey'].unique()) - set(data2['skey'].unique()))
stock_list =[str(i - 1000000).rjust(6, "0") + '.SH' if i < 2000000 else str(i - 2000000).rjust(6, "0") + '.SZ' for i in stock_list]
dd = str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:]
add1 = pd.DataFrame(columns=['date', 'stock_list'])
add1['stock_list'] = stock_list
add1['date'] = dd
add += [add1]
data2 = data2.rename(columns={'i_weight': 'weight'})
data2 = data2[['date', 'skey', 'index_id', 'index_name', 'weight']]

data2 = data2.drop_duplicates(keep='first')
add = pd.concat(add).reset_index(drop=True)
print('get index composition weight')
print(datetime.datetime.now() - startTm)

data3 = []
startTm = datetime.datetime.now()
stock_list = add['stock_list'].unique()    
for s in stock_list:
    start_date = add[add['stock_list'] == s]['date'].min()
    end_date = add[add['stock_list'] == s]['date'].max()
    add_data = w.wsd(s, "industry_CSRCcode12", start_date, end_date, "industryType=3;PriceAdj=F")
    if add_data.ErrorCode != 0:
        continue
    nd = pd.DataFrame(data=np.array(add_data.Data).T, columns=['Ind'])
    nd1 = pd.DataFrame(data=np.array(add_data.Times).T, columns=['DateTime'])
    nd = pd.concat([nd1, nd], axis=1)
    nd = nd[~nd['Ind'].isnull()]
    if nd.empty:
        continue
    else:
        nd['index_id'] = nd['Ind'].str[1:].astype(int)
        nd['date'] = nd['DateTime'].astype(str).apply(lambda x: x.replace('-', '')).astype(int)
        if s[-2:] == 'SZ':
            nd['skey'] = int(s[:-3]) + 2000000
        else:
            nd['skey'] = int(s[:-3]) + 1000000
        nd['weight'] = 0
        data3 += [nd]
data3 = pd.concat(data3).reset_index(drop=True)
print(datetime.datetime.now() - startTm)
print('get extra data')

data2 = pd.concat([data2, data3])
data2 = data2.sort_values(by=['date', 'skey', 'weight']).reset_index(drop=True)
if data2[data2.duplicated(['date', 'skey'], keep=False)].shape[0] != 0:
    display(data2[data2.duplicated(['date', 'skey'], keep=False)])
    data2 = data2.drop_duplicates(['date', 'skey'], keep='last').reset_index(drop=True)
assert(data2[data2['index_id'] < 100]['weight'].unique() == [0])

data2['index_id'] = np.where(data2.index_id <= 5, 3011030,np.where(data2.index_id <= 12, 3011031, \
                   np.where(data2.index_id == 13, 3030041,np.where(data2.index_id == 14, 3030042, \
                   np.where(data2.index_id == 15, 3030043,np.where(data2.index_id == 17, 3030044,\
                   np.where(data2.index_id == 18, 3030045,np.where(data2.index_id == 19, 3030046,\
                   np.where(data2.index_id == 20, 3030047,np.where(data2.index_id == 21, 3030048,\
                   np.where(data2.index_id == 22, 3030049,np.where(data2.index_id == 23, 3030050,\
                   np.where(data2.index_id == 24, 3030051,np.where(data2.index_id == 25, 3030052,\
                   np.where(data2.index_id == 26, 3030053,np.where(data2.index_id == 27, 3030054,\
                   np.where(data2.index_id == 28, 3030055,np.where(data2.index_id == 29, 3030056,\
                   np.where(data2.index_id == 30, 3030057,np.where(data2.index_id == 31, 3030058,\
                   np.where(data2.index_id == 32, 3030059,np.where(data2.index_id == 33, 3030060,\
                   np.where(data2.index_id == 34, 3030061,np.where(data2.index_id == 35, 3030062,\
                   np.where(data2.index_id == 36, 3030063,np.where(data2.index_id == 37, 3030064,\
                   np.where(data2.index_id == 38, 3030065,np.where(data2.index_id == 39, 3030066,\
                   np.where(data2.index_id == 40, 3030067,np.where(data2.index_id <= 43, 3011050,\
                   np.where(data2.index_id <= 46, 3011041,np.where(data2.index_id <= 50, 3011042,\
                   np.where(data2.index_id <= 52, 3011045,np.where(data2.index_id <= 60, 3011043,\
                   np.where(data2.index_id <= 62, 3030036,np.where(data2.index_id <= 65, 3011044,\
                   np.where(data2.index_id <= 69, 3011046,np.where(data2.index_id == 70, 3011047,\
                   np.where(data2.index_id <= 72, 3030037,np.where(data2.index_id <= 75, 3030038,\
                   np.where(data2.index_id <= 78, 3030039,np.where(data2.index_id <= 81, 3030040,\
                   np.where(data2.index_id == 82, 3011049,np.where(data2.index_id <= 84, 3030040,\
                   np.where(data2.index_id <= 89, 3011049,np.where(data2.index_id == 90, 3011050, data2['index_id']))))))))))))))))))))))))))))))))))))))))))))))
assert(data2['index_id'].min() > 100)
corrections = {3011030: 'AMAC 农林',    
               3011031: 'AMAC 采矿', 
               3011041: 'AMAC 公用', 
               3011042: 'AMAC 建筑', 
               3011043: 'AMAC 交运',    
               3011044: 'AMAC 信息', 
               3011045: 'AMAC 批零', 
               3011046: 'AMAC 金融', 
               3011047: 'AMAC 地产',    
               3011049: 'AMAC 文体', 
               3011050: 'AMAC 综企', 
               3030036: 'AMAC 餐饮',  
               3030037: 'AMAC 商务',    
               3030038: 'AMAC 科技', 
               3030039: 'AMAC 公共', 
               3030040: 'AMAC 社会',
               3030041: 'AMAC 农副', 
               3030042: 'AMAC 食品', 
               3030043: 'AMAC 饮料', 
               3030044: 'AMAC 纺织',    
               3030045: 'AMAC 服装', 
               3030046: 'AMAC 皮革', 
               3030047: 'AMAC 木材', 
               3030048: 'AMAC 家具',    
               3030049: 'AMAC 造纸', 
               3030050: 'AMAC 印刷', 
               3030051: 'AMAC 文教',  
               3030052: 'AMAC 石化',    
               3030053: 'AMAC 化学', 
               3030054: 'AMAC 医药', 
               3030055: 'AMAC 化纤',  
               3030056: 'AMAC 橡胶', 
               3030057: 'AMAC 矿物', 
               3030058: 'AMAC 钢铁', 
               3030059: 'AMAC 有色',    
               3030060: 'AMAC 金属', 
               3030061: 'AMAC 通用', 
               3030062: 'AMAC 专用', 
               3030063: 'AMAC 汽车',    
               3030064: 'AMAC 运输', 
               3030065: 'AMAC 电气', 
               3030066: 'AMAC 电子',  
               3030067: 'AMAC 仪表'}
data2['index_name'] = data2['index_id']
data2.index_name = data2.index_name.map(corrections)
data2 = data2[['date', 'skey', 'index_id', 'index_name', 'weight']]
try:
    assert(abs(data2.groupby(['date', 'index_id'])['weight'].sum() - 100).max() < 0.1)
except:
    print(data2.groupby(['date', 'index_id'])['weight'].sum())
write_memb_data(db1, 'index_memb', data2)